###  위험도

In [21]:
import pandas as pd
import numpy as np
import os 
import pandas as pds
from dask import dataframe
import re
import numpy as np
import seaborn as sbn
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import wordnet

In [22]:
train = pd.read_csv("train2.csv")
test= pd.read_csv("test2.csv")

In [23]:
len(test)

1418916

In [24]:
#eda

In [25]:
lit = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
def mask(tt):
    tt=tt.apply(lambda x: re.sub(r'(\\n)',' ',x))
    tt=tt.apply(lambda x: re.sub(r'[^a-zA-Zㄱ-ㅣ가-힣0-9:=\s\(\)./,\<\>]+',' ',x))
    #tt=tt.apply(lambda x: re.sub(r' ?(?P<note>[:=\(\)./,\<\>]) ?', ' \g<note> ', x))
    tt=tt.apply(lambda x: re.sub(r'[0-9]+',' ',x))
    tt=tt.apply(lambda x: re.sub(r"':/()",' ',x))
    tt=tt.apply(lambda x: re.sub(r':',' ',x))
    tt=tt.apply(lambda x: re.sub(r',',' ',x))
    # = tt.apply(lambda x: re.sub(r'(',' ',x))
    #t = tt.apply(lambda x: re.sub(r')',' ',x))
    tt=tt.apply(lambda x: re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',' ',x))
    for st in lit:
        st = " "+st + " "
        tt=tt.apply(lambda x: re.sub(st,' ',x))
    tt=tt.apply(lambda x: re.sub(r'\s+',' ',x))
    
    return tt

In [26]:
train['pre_log'] = mask(train.full_log)
test['pre_log'] = mask(test.full_log)

In [27]:
#포함되어 있는 단어 모음 생성
def count_word(data):
    tem = list(data['pre_log'].str.split(" "))
    all_word = []
    for word in tem:
        all_word.extend(word)
    words = pd.Series(all_word)
    return words.value_counts()

In [28]:
train_count = count_word(train)
test_count = count_word(test)

In [29]:
train_words = list(set(train_count.index))
test_words = list(set(test_count.index))

In [30]:
#영어 단어 + 3글자 이상인 경우만 True
from nltk.corpus import words
def check_words(word_list: list):
    re = [False] * len(word_list)
    for i,word in enumerate(word_list):
        if len(word) < 3:
            continue
        word = word.lower()
        if word in words.words():
            re[i] = True
    return re

word_list = ['Promiscuous', 'ab', 'nihongo', 'abstract', 'pedo','gid']

In [31]:
#check_words는 단어인지를 확인해주고 결과를 t/f로 반환
check_words(word_list)

[True, False, False, True, False, True]

In [32]:
train_tf = check_words(train_words)
test_tf = check_words(test_words)

In [33]:
#단어인지 확인해주는 딕셔너리 만들어주기 (train_isword,test_isword)
train_isword = dict()
test_isword = dict()

total_words = list(train_words)
test_words = list(test_words)

for i in range(len(train_words)):
    train_isword[train_words[i].lower()] = train_tf[i]
    
for i in range(len(test_words)):
    test_isword[test_words[i].lower()] = test_tf[i]

In [34]:
#train,test 적용

In [35]:
import copy

def checking_train(data):
    re = [False] * len(data)
    for i,word in enumerate(data):
        if train_isword[word]:
            re[i] = True
    return re


def checking_test(data):
    re = [False] * len(data)
    for i,word in enumerate(data):
        if test_isword[word]:
            re[i] = True
    return re



def cutt_train(data):
    data = data.lower()
    splited = data.split(" ")
    check = checking_train(splited)
    c = np.array(splited)
    real_words = list(c[check])    
    tem = " ".join(real_words)
    #tem = tem.lower() 
    return tem

def cutt_test(data):
    data = data.lower()
    splited = data.split(" ")
    check = checking_test(splited)
    c = np.array(splited)
    real_words = list(c[check])  
    tem = " ".join(real_words)
    #tem = tem.lower()
    return tem

In [36]:
train['cut'] = train['pre_log'].map(cutt_train)
test['cut'] = test['pre_log'].map(cutt_test)

In [37]:
del train['full_log']
del test['full_log']

del train['pre_log']
del test['pre_log']

In [38]:
train_bag = train['cut'].unique()
len(train_bag)

6544

In [39]:
#inornot을 통해서 train안에 같은 문장이 있으면 0 없으면 1
from tqdm import tqdm
inornot = np.zeros(len(test))
for i in tqdm(range(len(test))):
    tem = test.iloc[i]['cut']
    if tem not in train_bag:
        inornot[i] = 1

100%|██████████████████████████████████████████████████████████████████████| 1418916/1418916 [10:21<00:00, 2283.38it/s]


In [40]:
#모델

In [41]:
train.head(20)

,id,level,cut
0,0,0,type error warning collection level error erro...
1,1,0,action with response code type unavailable exc...
2,2,0,type error warning collection level error erro...
3,3,0,type error warning collection level error erro...
4,4,1,type audit arch success yes exit gid suid none...
5,5,1,type audit arch success yes exit gid suid none...
6,6,0,type error warning collection level error erro...
7,7,0,warn resurrect connection dead instance but go...
8,8,0,type error warning collection level error erro...
9,9,0,type log warning message error living share no...


In [42]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

#남은 단어가 없는 경우에는 'missing' 으로 대체해주었습니다.
train['cut'] = train['cut'].replace('','missing',regex=True)
test['cut'] = test['cut'].replace('','missing',regex=True)



train = train.fillna("missing")
test = test.fillna("missing")

train

,id,level,cut
0,0,0,type error warning collection level error erro...
1,1,0,action with response code type unavailable exc...
2,2,0,type error warning collection level error erro...
3,3,0,type error warning collection level error erro...
4,4,1,type audit arch success yes exit gid suid none...
...,...,...,...
472967,472967,0,error
472968,472968,1,type audit arch success yes exit gid suid none...
472969,472969,0,type log error task manager message poll for w...
472970,472970,0,type error warning collection level error erro...


In [43]:
train_text=list(train['cut'])
train_level=np.array(train['level'])

test_text=list(test['cut'])

In [44]:
vectorizer=CountVectorizer(analyzer="word", max_features=20000)
train_features=vectorizer.fit_transform(train_text)
test_features=vectorizer.transform(test_text)

In [45]:
forest=RandomForestClassifier(n_estimators=100,random_state = 1 )
forest.fit(train_features,train_level)

RandomForestClassifier(random_state=1)

In [46]:
results=forest.predict(test_features)
results_proba=forest.predict_proba(test_features)

In [47]:
#Validation을 통한 threshold 조절
results[np.where((np.max(results_proba, axis=1)<0.5) & (results == 0))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.5) & (results == 1))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 2))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.95) & (results == 3))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 4))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 5))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 6))[0]]=7


In [48]:
#Validation을 통한 threshold 조절
results[np.where((np.max(results_proba, axis=1)<0.94001) & (np.max(results_proba, axis=1) > 0.93999) & (results == 1))[0]]=7 
results[np.where((np.max(results_proba, axis=1)<0.611657) & (np.max(results_proba, axis=1) > 0.6116568) & (results == 0))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.571657) & (np.max(results_proba, axis=1) > 0.5716568) & (results == 0))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.68001) & (np.max(results_proba, axis=1) > 0.67999) & (results == 5))[0]]=7 

In [49]:
#결과저장

In [56]:
submission=pd.read_csv('sample_submission2.csv')
submission['level']=results
submission['level'].value_counts()

0    1002369
1     396443
3      12813
5       6418
7        780
2         34
4         34
6         25
Name: level, dtype: int64

In [57]:
submission.to_csv("tem_answer.csv",index = False)

In [58]:
#추가작업

In [59]:
submission['inornot'] = pd.Series(inornot)

In [60]:
cal = submission[(submission['level']==7) & (submission['inornot'] == 0)].index
not7_id = list(test.iloc[cal]['id'])
pd.Series(not7_id).to_csv("not7_id.csv",index= False)

In [64]:
train=pd.read_csv('train2.csv')
test=pd.read_csv('test2.csv')


In [65]:
train['full_log']=train['full_log'].str.replace(r'[0-9]', '<num>')
test['full_log']=test['full_log'].str.replace(r'[0-9]', '<num>')

<ipython-input-65-2f9bd1abbc44>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train['full_log']=train['full_log'].str.replace(r'[0-9]', '<num>')
<ipython-input-65-2f9bd1abbc44>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  test['full_log']=test['full_log'].str.replace(r'[0-9]', '<num>')


In [66]:
train = train.fillna("missing")
test = test.fillna("missing")

In [67]:
train_text=list(train['full_log'])
train_level=np.array(train['level'])

test_text=list(test['full_log'])
#ids=list(test['id'])

In [68]:
vectorizer=TfidfVectorizer(analyzer="word", max_features=20000)
train_features=vectorizer.fit_transform(train_text)
test_features=vectorizer.transform(test_text)

In [69]:
forest=RandomForestClassifier(n_estimators=100,random_state = 1 )
forest.fit(train_features,train_level)

RandomForestClassifier(random_state=1)

In [70]:
results=forest.predict(test_features)
results_proba=forest.predict_proba(test_features)

In [72]:
submission=pd.read_csv('sample_submission2.csv')
submission['level']=results
submission['level'].value_counts()

0    1002770
1     396532
3      12997
5       6524
2         34
4         34
6         25
Name: level, dtype: int64

In [73]:
not7_id = pd.read_csv('not7_id.csv')
not7_id = list(not7_id['0'])

In [74]:
#위에서 잡은 7 합치기 & train에 있는 경우는 사용하지 않기
only = pd.read_csv("tem_answer.csv")
le7 = list(only[only['level']==7]['id'])
le7_update = list((set(le7))-set(not7_id))
print(len(le7))
print(len(le7_update))
idx = submission[submission['id'].isin(le7_update)].index
submission.iloc[idx] = 7
submission['level'].value_counts()

780
581


0    1002512
1     396348
3      12915
5       6467
7        581
2         34
4         34
6         25
Name: level, dtype: int64

In [75]:
who = submission.level
submission=pd.read_csv('sample_submission.csv')
submission['level'] = who
submission['level'].value_counts()


0    30752
1    12200
3      394
5      209
7       17
2        2
4        1
6        1
Name: level, dtype: int64

submission.to_csv("581_final_answer.csv",index = False)